## 21.3

The Maxwell-Boltzman distribution, often called just a Maxwellian distribution, gives the distribution of speeds of particles in a gas by the formula

$$f(v) = \sqrt{\left(\frac{m}{2 \pi kT}\right)^3}\, 4\pi v^2 e^{- \frac{mv^2}{2kT}}$$

where $m$ is the mass of the particles, $T$ is the temperature, and $k$ is the Boltzmann constant.  Consider a gas of deuterium at $kT = 1$ keV $= 1.60218 \times 10^{-16}$ J.  Sample particle speeds from the Maxwellian using rejection sampling.  From your sampled points, compute the mean speed and the square-root of the mean speed squared (i.e., compute the mean value of the speed squared and then take the square root, aka the root-mean square speed).   The mean speed should be 

$$\int_0^\infty dv\, v f(v) = \sqrt{\frac{8kT}{\pi m}},$$

and the root-mean square speed should be

$$\sqrt{ \int_0^\infty dv\, v^2 f(v)} = \sqrt{\frac{3kT}{m}}.$$

Compute these quantities using sample numbers of $N=10,10^2,10^4,10^6$ and discuss your results.

## Solution

First, we must define a function that represents the Maxwellian distribution for sampling. When performing rejection sampling, it is often useful to plot the distribution over a wide variety of energies to determine a good range to sample in. Looking at the first plot below, it is clear why the energy range chosen was $E = [0.0,1 \times 10^6]$. In addition, the maximum height chosen was determined by the max of the function in the range chosen. It is acceptable to chose a wider range, but it will require a larger number of samples for the same result. Bounding the samples to a reasonable range is a good choice.

$\br$We will then loop through each number of samples requested. Empty lists are created that will be filled with the rejected and accepted samples. Another loop is then created that will loop through the desired number of samples. A $v$ is then sampled in the range above from a uniform distribution, and the same done in the $y$ direction. Each sample is checked and either accepted or rejected.

$\br$After the sampling is complete, the mean speed and square-root of the mean speed are calculated and compared to the exact solution.

$\br$Note that the first three samplings are plotted as a visual aid, and that these plots are not required for full credit.$\br$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Define constants
kT = 1.60218E-16 # [J]
Na = 6.022E23 # [atoms/mole]
m = 2.01410178E-3/Na # [g/atom]

# Define distribution
def f(v,m,kT):
    return np.sqrt((m/(2*np.pi*kT))**3)*4*np.pi*v**2*np.exp(-m*v**2/(2*kT))

# Define min and max velocities
min_v = 0.0
max_v = 1.0e6

# Determine max in range
numvs = 10**5
vs = np.linspace(min_v,max_v,numvs)
max_dist = np.max(f(vs,m,kT))

# Determine exact mean and root-mean square
meanExact = np.sqrt(8*kT/(np.pi*m))
rootMeanExact = np.sqrt(3*kT/m)
print('The exact exact mean speed is', '%.5e' % meanExact,'m/s')
print('The exact root-mean square speed is', '%.5e' % rootMeanExact,'m/s\n')

# Sample numbers to loop through
Ns = 10**(np.array([1,2,4,6]))
for N in Ns:
    # Fill arrays
    accepted_v = []
    accepted_y = []
    rejected_v = []
    rejected_y = []
    
    # Sample N particles
    for sample in range(N):
        # Sample v
        v = np.random.uniform(min_v,max_v,1)
        y = np.random.uniform(0,max_dist,1)
        rel_prob = f(v,m,kT)
        # Accept or reject and append
        if y <= rel_prob:
            accepted_v.append(v)
            accepted_y.append(y)
        else:
            rejected_v.append(v)
            rejected_y.append(y)
            
    # Numpy array please
    accepted_v = np.array(accepted_v)
    accepted_y = np.array(accepted_y)
    rejected_v = np.array(rejected_v)
    rejected_y = np.array(rejected_y)

    # Plot (not required)
    # But not for 10^6 particles--too many
    if N < 10**6:
        plt.title('Rejection sampling of $10^' + str(np.log10(N))[0] + '$ particles')
        plt.plot(accepted_v,accepted_y,'+',color='green',label='Accepted samples')
        plt.plot(rejected_v,rejected_y,'x',color='red',label='Rejected samples')
        plt.plot(vs,f(vs,m,kT),label=r'$f(v)$',color='black',linewidth=2)
        plt.plot(vs,max_dist*np.ones(numvs),'--',color='black',label='Maximum',linewidth=2)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xlabel('Particle velocity (m/s)')
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        plt.show()
    
    print('For N = 10^',str(np.log10(N))[0],'particles:')
    
    # Determine required means
    mean = np.mean(accepted_v)
    meanError = np.abs((mean-meanExact)/meanExact)*100
    rootMean = np.sqrt(np.mean(accepted_v**2))
    rootMeanError = np.abs((rootMean-rootMeanExact)/rootMeanExact)*100
    
    # Annd print
    print('Mean speed:', '%.5e' % mean,'m/s with %.2f' % meanError + '% error')
    print('Root-mean square speed:', '%.5e' % rootMean,'m/s with %.2f' % rootMeanError + '% error')

Notice how the errors begin to converge, and that when $10^6$ particles are ran we end up with a rather good set of samples that exhibit the mean behavior very well. 

## Chapter 21 Short Exercise 2

Modify the shielding code to consider neutrons of a single energy impinging on the shield and to tally the energy of the absorbed neutrons.  Assume the neutrons are all 2.5 MeV and are produced from the fusion of deuterium. Plot the distribution of transmitted and absorbed neutrons with a large enough number of sample particles.

## Solution

First, we will start with the $\texttt{slab\_reactor}$ function from the Chapter 21 notes. Changes are made below such that no sampling is done over energy. Instead, all samples are assigned the same energy. The function is then renamed $\texttt{slab\_reactor\_mod}$.

$\br$In addition, code is copied that imports the energy-dependent cross sections from a csv format for input to the function. 

$\br$As it was chosen to simulate 1 million particles, the function was also modifed to print out the status every 100,000 particles.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def slab_reactor_mod(Sig_s,Sig_a,E,thickness,density,A,N,isotropic=False):
    """Compute the fraction of neutrons that leak through a slab and
    have a constant incident energy
    Inputs:
    Sig_s: The scattering macroscopic x-section array in form Energy, X-sect
    Sig_a: The absorption macroscopic x-section
    E: incident neutron energy
    thickness: Width of the slab
    density:  density of material in atoms per cc
    A:         atomic weight of shield
    N:         Number of neutrons to simulate
    isotropic: Are the neutrons isotropic or a beam
    
    Returns:
    transmission: energies of neutrons that leak through
    created: energies of neutrons that were born
    """
    
    alpha = (A-1.0)**2/(A+1.0)**2
    transmission = []
    created = []
    absorbed = []
    N = int(N)
    for i in range(N):
        #sample direction
        if (isotropic):
            mu = np.random.random(1)
        else:
            mu = 1.0
        #energy is constant
        energy = E
        #initial position is 0
        x=0
        created.append(E)
        alive = 1
        while (alive):
            #get distance to collision
            scat_index = energy_lookup(Sig_s[0,:],energy)
            abs_index = energy_lookup(Sig_a[0,:],energy)
            cur_scat = Sig_s[1,scat_index]
            cur_abs = Sig_a[1,abs_index]
            Sig_t = cur_scat + cur_abs
            l = -np.log(1-np.random.random(1))/Sig_t
            #move particle
            x += l*mu
            #still in the slab
            if (x>thickness):
                transmission.append(energy)
                alive = 0 
            elif (x<0):
                alive = 0
            else:
                #scatter or absorb
                if (np.random.random(1) < cur_scat/Sig_t):
                    #scatter, pick new mu and energy
                    mu = np.random.uniform(-1,1,1)
                    energy = np.random.uniform(alpha*energy,energy,1)
                else: #absorbed 
                    absorbed.append(energy)
                    alive = 0
        if i%100000 == 0:
            print('Simulated',i,'particles')
    return transmission, created, absorbed

def energy_lookup(data_set, inp_energy):
    """look up energy in a data set and return the nearest energy in the table
    Input:
    data_set: a vector of energies
    inp_energy: the energy to lookup
    Output:
    index: the index of the nearest neighbor in the table
    """
    #argmin returns the indices of the smallest members of an array
    #here we'll look for the minimum difference between the input energy and the table
    index = np.argmin(np.fabs(data_set-inp_energy))
    return index

# Import energy dependent cross-sections for Pb-208
import csv
lead_s = [] #create a blank list for the x-sects
lead_s_energy = [] #create a blank list for the x-sects energies
#this loop will only execute if the file opens
with open('pb_scat.csv') as csvfile:
    pbScat = csv.reader(csvfile)
    for row in pbScat: #have for loop that loops over each line
        lead_s.append(float(row[1]))
        lead_s_energy.append(float(row[0]))
lead_scattering = np.array([lead_s_energy,lead_s])
lead_abs = [] #create a blank list for the x-sects
lead_abs_energy = [] #create a blank list for the x-sects energie
#this loop will only execute if the file opens
with open('pb_radcap.csv') as csvfile:
    pbAbs = csv.reader(csvfile)
    for row in pbAbs: #have for loop that loops over each line
        lead_abs.append(float(row[1]))
        lead_abs_energy.append(float(row[0]))
lead_absorption = np.array([lead_abs_energy,lead_abs])

# Define necessary inputs
E = 2.5E6 # [MeV]
N = 1e6 # [particles]
A = 208 # [g/mol]
Na = 6.022E23 # [atoms/mol]
density = 11.34/A*Na # [atoms/cm^3]
thickness = 5 # [cm]

# Run!
transmission,created,absorbed = slab_reactor_mod(lead_scattering,lead_absorption,
                                                 E,thickness,density,A,N,isotropic=True)

Now that we have our data, we will plot. First will come a plot of the source particles. Note that this plot is optional, but it is done to verify that all of our particles are being born at 2.5 MeV.$\br$

In [ ]:
# Plot created particles
plt.hist(np.array(created)/1.e6,bins = 10 ** np.linspace(np.log10(0.001), np.log10(10), 200))
plt.gca().set_xscale("log")
plt.gca().set_yscale("log")
plt.xlabel("Energy (MeV)")
plt.ylabel('Particles')
plt.title("Created particles (optional plot)")
plt.show()

Great--2.5 MeV it is.

$\br$Last, we will plot the transmitted and absorbed particles.$\br$

In [ ]:
# Plot transmitted particles
plt.hist(np.array(transmission)/1.e6,bins = 10 ** np.linspace(np.log10(0.001), np.log10(10), 200))
plt.gca().set_xscale("log")
plt.gca().set_yscale("log")
plt.xlabel("Energy (MeV)")
plt.ylabel('Particles')
plt.title("Transmitted particles")
plt.show()

# Plot absorbed particles
plt.hist(np.array(absorbed)/1.e6,bins = 10 ** np.linspace(np.log10(0.001), np.log10(10), 200))
plt.gca().set_xscale("log")
plt.gca().set_yscale("log")
plt.xlabel("Energy (MeV)")
plt.ylabel('Particles')
plt.title("Absorbed particles")
plt.show()

## Pure Plutonium Reactor: Part 1

In Chapter 18 we defined a pulsed reactor made of pure plutonium with the following cross-sections from the report $\it{Reactor~Physics~Constants}$, ANL-5800:

For the density of plutonium use $19.74$ g/cm$^3$; you may assume that $\sigma_\mathrm{t} \approx \sigma_\mathrm{tr}.$ 

$\br$Compute $k_\mathrm{eff}$ for a slab of thickness 7 cm made from pure plutonium-239 using fission cycles and the fission matrix method. 

## Solution

The file $\texttt{ch23.py}$ will be imported, as it contains all of the functions present in the Chapter 23 notes.

$\br$We will first define all of the given information for use in the later functions.

$\br$As $\sigma_\mathrm{t} \approx \sigma_\mathrm{tr}$, we will easily compute the scattering cross section as

$$\Sigma_s = \Sigma_\mathrm{tr} - \Sigma_\mathrm{a}.$$

In [ ]:
# NUEN 329
# Pure Plutonium Reactor: Part 1

import numpy as np
from ch23 import *

# Define cool numbers
Na = 6.022E23 # [atoms/mol]

# Define plutonium constants
sigf = 1.85E-24 # [cm^2]
siga = 2.11E-24 # [cm^2]
sigtr = 6.8E-24 # [cm^2]
nu = 2.98 # [n/fiss]
rho = 19.74 # [g/cm^3]
m = 239.05429 # [g]

# Convert plutonium cross sections
Sigf = Na*rho/m*sigf # [1/cm]
Siga = Na*rho/m*siga # [1/cm]
Sigtr = Na*rho/m*sigtr # [1/cm]
Sigs = Sigtr - Siga # [1/cm]

$\br$We will start with the fission cycle method, using the $\texttt{homog\_slab\_k}$ function. All that is necessary here is to define the input values for the function. Here, we will run 10 inactive cycles and 50 active cycles of 20,000 neutrons. A smaller number of cycles and neutrons are acceptable, but the solution looks a bit better otherwise.

$\br$After running the function, we will plot the resulting values of $k$ and determine the average value using an exceprt from the lecture notes.$\br$

In [ ]:
# Solve using fission cycles
N = 20000
thickness = 7.0 # [cm]
inactive_cycles = 10
active_cycles = 50
k = homog_slab_k(N,Sigtr,Sigs,Sigf,nu,thickness,inactive_cycles,active_cycles)

# Plot
plt.plot(k)
plt.title(str(np.mean(k[inactive_cycles:(active_cycles+inactive_cycles)]))+
        "$\pm$" + str(np.std(k[inactive_cycles:(active_cycles+inactive_cycles)])))
plt.xlabel("Cycle")
plt.ylabel("k")
plt.plot([inactive_cycles,inactive_cycles],[np.min(k)*.9,np.max(k)*1.1])
plt.axis([0,k.size,np.min(k)*.9,np.max(k)*1.1])
plt.show()

Next, we will use the fission matrix method. The $\texttt{fission\_matrix}$ function will be used to define the matrix, and the NumPy function $\texttt{linalg.eig}$ will be used to solve for the eigenvalues of the matrix.

$\br$We will look at the entires in the fission matrix, and plot the eigenvalues (also taken directly from the lecture notes).$\br$

In [ ]:
# Solve using fission matrix
N = 100000
Nx = 50
H,Xmid = fission_matrix(N,Sigtr,Sigs,Sigf,nu,thickness,Nx)

# Plot matrix
plt.pcolormesh(H)
plt.title("Entries in matrix $\mathbf{H}$")
plt.colorbar()
plt.show()
ks, v = np.linalg.eig(H)

# Plot eigenvalues
plt.plot(np.real(ks),np.imag(ks),'o')
plt.title("$k_\mathrm{eff} = $" + str(np.real(np.max(ks))))
plt.xlabel("Real part of k")
plt.ylabel("Imaginary part of k")
plt.show()

Note that both methods resulted in a similar value of $k_\mathrm{eff}$.